##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aproximação de matrizes com as APIs principais (Core)

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/core/matrix_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver no TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/matrix_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/matrix_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/core/matrix_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

## Introdução

Este notebook usa as [APIs de baixo nível do TensorFlow Core](https://www.tensorflow.org/guide/core) para mostrar os recursos do TensorFlow como uma plataforma de computação científica de alto desempenho. Veja a [Visão geral das APIs Core](https://www.tensorflow.org/guide/core) para saber mais sobre o TensorFlow Core e seus casos de uso pretendidos.

Este tutorial explora a técnica de [decomposição em valores singulares](https://developers.google.com/machine-learning/recommendation/collaborative/matrix) (SVD) e suas aplicações para problemas de aproximação de posto baixo. O SVD é usado para fatorar matrizes reais ou complexas e tem uma variedade de casos de uso em ciência de dados, como compactação de imagens. As imagens para este tutorial vêm do projeto [Imagen](https://imagen.research.google/) do Google Brain. 

> ![svd_intro](http://tensorflow.org/images/core/svd_intro.png)

## Configuração

In [ ]:
import matplotlib
from matplotlib.image import imread
from matplotlib import pyplot as plt
import requests
# Preset Matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [16, 9]

In [ ]:
import tensorflow as tf
print(tf.__version__)

## Fundamentos do SVD

A decomposição em valores singulares de uma matriz, ${\mathrm{A}}$, é determinada pela seguinte fatoração:

$${\mathrm{A}} = {\mathrm{U}} \Sigma {\mathrm{V}}^T$$

onde

- $\underset{m \times n}{\mathrm{A}}$: matriz de entrada onde $m \geq n$
- $\underset{m \times n}{\mathrm{U}}$: matriz ortogonal, ${\mathrm{U}}^T{\mathrm{U}} = {\mathrm{I}}$, com cada coluna, $u_i$, denotando um vetor singular esquerdo de ${\mathrm{A}}$
- $\underset{n \times n}{\Sigma}$: matriz diagonal com cada entrada diagonal, $\sigma_i$, denotando um valor singular de ${\mathrm{A}}$
- $\underset{n \times n}{{\mathrm{V}}^T}$: matriz ortogonal, ${\mathrm{V}}^T{\mathrm{V}} = {\mathrm{I}}$, com cada linha, $v_i$, denotando um vetor singular direito de ${\mathrm{A}}$

Quando $m <n$, ${\mathrm{U}}$ e $\Sigma$  ambos têm dimensão $(m \times m)$, e ${\mathrm{V}}^T$ tem dimensão $(m \vezes n)$.

> ![svd_full](http://tensorflow.org/images/core/svd_full.png)

O pacote de álgebra linear do TensorFlow tem uma função, `tf.linalg.svd` , que pode ser usada para calcular a decomposição em valores singulares de uma ou mais matrizes. Comece definindo uma matriz simples e calculando sua fatoração SVD.


In [ ]:
A = tf.random.uniform(shape=[40,30])
# Compute the SVD factorization
s, U, V = tf.linalg.svd(A)
# Define Sigma and V Transpose
S = tf.linalg.diag(s)
V_T = tf.transpose(V)
# Reconstruct the original matrix
A_svd = U@S@V_T
# Visualize 
plt.bar(range(len(s)), s);
plt.xlabel("Singular value rank")
plt.ylabel("Singular value")
plt.title("Bar graph of singular values");

A função `tf.einsum` pode ser usada para calcular diretamente a reconstrução da matriz das saídas de `tf.linalg.svd` .

In [ ]:
A_svd = tf.einsum('s,us,vs -> uv',s,U,V)
print('\nReconstructed Matrix, A_svd', A_svd)

## Aproximação de posto baixo com o SVD

O posto de uma matriz, ${\mathrm{A}}$, é determinado pela dimensão do espaço vetorial representado por suas colunas. O SVD pode ser usado para aproximar uma matriz com posto inferior, o que acaba diminuindo a dimensionalidade dos dados necessários para armazenar as informações representadas pela matriz.

A aproximação de posto r de ${\mathrm{A}}$ em termos de SVD é definida pela fórmula:

$${\mathrm{A_r}} = {\mathrm{U_r}} \Sigma_r {\mathrm{V_r}}^T$$

onde

- $\underset{m \times r}{\mathrm{U_r}}$: matriz composta pelas primeiras $r$ colunas de ${\mathrm{U}}$
- $\underset{r \times r}{\Sigma_r}$: matriz diagonal composta pelos primeiros $r$ valores singulares em $\Sigma$
- $\underset{r \times n}{\mathrm{V_r}}^T$: matriz composta pelas primeiras $r$ linhas de ${\mathrm{V}}^T$

> ![svd_approx](http://tensorflow.org/images/core/svd_approx.png)

Comece escrevendo uma função para calcular a aproximação de posto r de uma determinada matriz. Este procedimento de aproximação de baixo posto é usado para compressão de imagens; portanto, também é útil calcular os tamanhos de dados físicos para cada aproximação. Para simplificar, suponha que o tamanho dos dados para uma matriz aproximada de posto r seja igual ao número total de elementos necessários para calcular a aproximação. Em seguida, escreva uma função para visualizar a matriz original, $\mathrm{A}$ sua aproximação de posto r, $\mathrm{A}_r$ e a matriz de erro, $|\mathrm{A} - \mathrm{A} _r|$.

In [ ]:
def rank_r_approx(s, U, V, r, verbose=False):
  # Compute the matrices necessary for a rank-r approximation
  s_r, U_r, V_r = s[..., :r], U[..., :, :r], V[..., :, :r] # ... implies any number of extra batch axes
  # Compute the low-rank approximation and its size
  A_r = tf.einsum('...s,...us,...vs->...uv',s_r,U_r,V_r)
  A_r_size = tf.size(U_r) + tf.size(s_r) + tf.size(V_r)
  if verbose:
    print(f"Approximation Size: {A_r_size}")
  return A_r, A_r_size

def viz_approx(A, A_r):
  # Plot A, A_r, and A - A_r
  vmin, vmax = 0, tf.reduce_max(A)
  fig, ax = plt.subplots(1,3)
  mats = [A, A_r, abs(A - A_r)]
  titles = ['Original A', 'Approximated A_r', 'Error |A - A_r|']
  for i, (mat, title) in enumerate(zip(mats, titles)):
    ax[i].pcolormesh(mat, vmin=vmin, vmax=vmax)
    ax[i].set_title(title)
    ax[i].axis('off')

In [ ]:
print(f"Original Size of A: {tf.size(A)}")
s, U, V = tf.linalg.svd(A)

In [ ]:
# Rank-15 approximation
A_15, A_15_size = rank_r_approx(s, U, V, 15, verbose = True)
viz_approx(A, A_15)

In [ ]:
# Rank-3 approximation
A_3, A_3_size = rank_r_approx(s, U, V, 3, verbose = True)
viz_approx(A, A_3)

Como esperado, usar postos mais baixos resulta em aproximações menos precisas. No entanto, a qualidade dessas aproximações de baixo posto costuma ser boa o suficiente em cenários do mundo real. Observe também que o principal objetivo da aproximação de baixo posto com SVD é reduzir a dimensionalidade dos dados, mas não reduzir o espaço em disco ocupado pelos próprios dados. No entanto, à medida em que são usadas matrizes de entrada de dimensões superiores, muitas aproximações de baixo posto também acabam se beneficiando do tamanho reduzido dos dados. Esse benefício de redução é o motivo pelo qual o processo é aplicável para problemas de compactação de imagem.

## Carregamento de imagens

A imagem a seguir está disponível na página inicial [do Imagen](https://imagen.research.google/). Imagen é um modelo de difusão de texto para imagem desenvolvido pela equipe Brain do Google Research. Uma IA criou esta imagem com base no prompt a seguir: "Uma foto de um cachorro Corgi andando de bicicleta na Times Square. Ele está usando óculos escuros e um chapéu de praia". Não é legal? Você também pode alterar a URL abaixo para qualquer link .jpg para carregar qualquer imagem personalizada de sua escolha.

Comece lendo e visualizando a imagem. Depois de ler um arquivo JPEG, o Matplotlib gera uma matriz, ${\mathrm{I}}$, de formato $(m \times n \times 3)$ que representa uma imagem bidimensional com 3 canais de cores para vermelho, verde e azul respectivamente.

In [ ]:
img_link = "https://imagen.research.google/main_gallery_images/a-photo-of-a-corgi-dog-riding-a-bike-in-times-square.jpg"
img_path = requests.get(img_link, stream=True).raw
I = imread(img_path, 0)
print("Input Image Shape:", I.shape)

In [ ]:
def show_img(I):
  # Display the image in matplotlib
  img = plt.imshow(I)
  plt.axis('off')
  return

In [ ]:
show_img(I)

## O algoritmo de compressão de imagem

Agora, use o SVD para calcular aproximações de posto baixo da imagem de exemplo. Lembre-se de que a imagem tem o formato $(1024 \times 1024 \times 3)$ e que a teoria SVD só se aplica a matrizes bidimensionais. Isto significa que a imagem de exemplo deve ser loteada em 3 matrizes de tamanho igual que correspondem a cada um dos 3 canais de cores. Isto pode ser feito transpondo a matriz para o formato $(3 \times 1024 \times 1024)$. Para visualizar claramente o erro de aproximação, redimensione os valores RGB da imagem de $[0,255]$ para $[0,1]$. Lembre-se de truncar os valores aproximados para que fiquem dentro desse intervalo antes de visualizá-los. A função `tf.clip_by_value` é útil para isso.

In [ ]:
def compress_image(I, r, verbose=False):
  # Compress an image with the SVD given a rank 
  I_size = tf.size(I)
  print(f"Original size of image: {I_size}")
  # Compute SVD of image
  I = tf.convert_to_tensor(I)/255
  I_batched = tf.transpose(I, [2, 0, 1]) # einops.rearrange(I, 'h w c -> c h w')
  s, U, V = tf.linalg.svd(I_batched)
  # Compute low-rank approximation of image across each RGB channel
  I_r, I_r_size = rank_r_approx(s, U, V, r)
  I_r = tf.transpose(I_r, [1, 2, 0]) # einops.rearrange(I_r, 'c h w -> h w c')
  I_r_prop = (I_r_size / I_size)
  if verbose:
    # Display compressed image and attributes
    print(f"Number of singular values used in compression: {r}")
    print(f"Compressed image size: {I_r_size}")
    print(f"Proportion of original size: {I_r_prop:.3f}")
    ax_1 = plt.subplot(1,2,1)
    show_img(tf.clip_by_value(I_r,0.,1.))
    ax_1.set_title("Approximated image")
    ax_2 = plt.subplot(1,2,2)
    show_img(tf.clip_by_value(0.5+abs(I-I_r),0.,1.))
    ax_2.set_title("Error")
  return I_r, I_r_prop

Agora, compute aproximações de posto r para os seguintes postos: 100, 50, 10

In [ ]:
I_100, I_100_prop = compress_image(I, 100, verbose=True)

In [ ]:
I_50, I_50_prop = compress_image(I, 50, verbose=True)

In [ ]:
I_10, I_10_prop = compress_image(I, 10, verbose=True)

## Computando aproximações

Há uma variedade de métodos interessantes para medir a eficácia e ter mais controle sobre as aproximações matriciais.

### Fator de compressão vs posto

Para cada uma das aproximações acima, observe como os tamanhos dos dados mudam com o posto.

In [ ]:
plt.figure(figsize=(11,6))
plt.plot([100, 50, 10], [I_100_prop, I_50_prop, I_10_prop])
plt.xlabel("Rank")
plt.ylabel("Proportion of original image size")
plt.title("Compression factor vs rank");

Com base nesse gráfico, há uma relação linear entre o fator de compressão de uma imagem aproximada e seu posto. Para explorar isso ainda mais, lembre-se de que o tamanho dos dados de uma matriz aproximada, ${\mathrm{A}}_r$, é definido como o número total de elementos necessários para sua computação. As seguintes equações podem ser usadas para encontrar a relação entre o fator de compressão e o posto:

$$x = (m \times r) + r + (r \times n) = r \times (m + n + 1)$$

$$c = \large \frac{x}{y} = \frac{r \times (m + n + 1)}{m \times n}$$

onde

- $x$: tamanho de ${\mathrm{A_r}}$
- $y$: tamanho de ${\mathrm{A}}$
- $c = \frac{x}{y}$: fator de compressão
- $r$: posto da aproximação
- $m$ e $n$: dimensões de linha e coluna de ${\mathrm{A}}$

Para encontrar o posto, $r$, que é necessário para comprimir uma imagem para um fator desejado, $c$, a equação acima pode ser reorganizada para a solução de $r$:

$$r = ⌊{\large\frac{c \times m \times n}{m + n + 1}}⌋$$

Observe que esta fórmula independe da dimensão do canal de cores, pois cada uma das aproximações RGB não afeta uma à outra. Agora, escreva uma função para comprimir uma imagem de entrada dado um fator de compressão desejado.

In [ ]:
def compress_image_with_factor(I, compression_factor, verbose=False):
  # Returns a compressed image based on a desired compression factor
  m,n,o = I.shape
  r = int((compression_factor * m * n)/(m + n + 1))
  I_r, I_r_prop = compress_image(I, r, verbose=verbose)
  return I_r

Comprima uma imagem em 15% de seu tamanho original.

In [ ]:
compression_factor = 0.15
I_r_img = compress_image_with_factor(I, compression_factor, verbose=True)

### Soma cumulativa de valores singulares

A soma cumulativa de valores singulares pode ser um indicador útil para a quantidade de energia capturada por uma aproximação de posto r. Visualize a proporção cumulativa de valores singulares de RGB médio na imagem de amostra. A função `tf.cumsum` pode ser útil para esse fim.

In [ ]:
def viz_energy(I):
  # Visualize the energy captured based on rank
  # Computing SVD
  I = tf.convert_to_tensor(I)/255
  I_batched = tf.transpose(I, [2, 0, 1]) 
  s, U, V = tf.linalg.svd(I_batched)
  # Plotting average proportion across RGB channels 
  props_rgb = tf.map_fn(lambda x: tf.cumsum(x)/tf.reduce_sum(x), s)
  props_rgb_mean = tf.reduce_mean(props_rgb, axis=0)
  plt.figure(figsize=(11,6))
  plt.plot(range(len(I)), props_rgb_mean, color='k')
  plt.xlabel("Rank / singular value number")
  plt.ylabel("Cumulative proportion of singular values")
  plt.title("RGB-averaged proportion of energy captured by the first 'r' singular values")

In [ ]:
viz_energy(I)

Parece que mais de 90% da energia desta imagem foi capturada nos primeiros 100 valores singulares. Agora, escreva uma função para comprimir uma imagem de entrada dado um fator de retenção de energia desejado.

In [ ]:
def compress_image_with_energy(I, energy_factor, verbose=False):
  # Returns a compressed image based on a desired energy factor
  # Computing SVD
  I_rescaled = tf.convert_to_tensor(I)/255
  I_batched = tf.transpose(I_rescaled, [2, 0, 1]) 
  s, U, V = tf.linalg.svd(I_batched)
  # Extracting singular values
  props_rgb = tf.map_fn(lambda x: tf.cumsum(x)/tf.reduce_sum(x), s)
  props_rgb_mean = tf.reduce_mean(props_rgb, axis=0)
  # Find closest r that corresponds to the energy factor
  r = tf.argmin(tf.abs(props_rgb_mean - energy_factor)) + 1
  actual_ef = props_rgb_mean[r]
  I_r, I_r_prop = compress_image(I, r, verbose=verbose)
  print(f"Proportion of energy captured by the first {r} singular values: {actual_ef:.3f}")
  return I_r

Comprima uma imagem para reter 75% de sua energia.

In [ ]:
energy_factor = 0.75
I_r_img = compress_image_with_energy(I, energy_factor, verbose=True)

### Erro e valores singulares

Existe também um relacionamento interessante entre o erro de aproximação e os valores singulares. Acontece que o quadrado da norma de Frobenius da aproximação é igual à soma dos quadrados de seus valores singulares que foram deixados de fora:

$${||A - A_r||}^2 = \sum_{i=r+1}^{R}σ_i^2$$

Teste esse relacionamento com uma aproximação de posto 10 da matriz de exemplo no início deste tutorial.

In [ ]:
s, U, V = tf.linalg.svd(A)
A_10, A_10_size = rank_r_approx(s, U, V, 10)
squared_norm = tf.norm(A - A_10)**2
s_squared_sum = tf.reduce_sum(s[10:]**2)
print(f"Squared Frobenius norm: {squared_norm:.3f}")
print(f"Sum of squared singular values left out: {s_squared_sum:.3f}")

## Conclusão

Este notebook apresentou o processo de implementação da decomposição em valores singulares com o TensorFlow e sua aplicação para escrever um algoritmo de compactação de imagens. Aqui estão mais algumas dicas que podem ser úteis:

- As [APIs Core do TensorFlow](https://www.tensorflow.org/guide/core) podem ser utilizadas para uma variedade de casos de uso de computação científica de alto desempenho.
- Para saber mais sobre as funcionalidades de álgebra linear do TensorFlow, veja os documentos do [módulo linalg](https://www.tensorflow.org/api_docs/python/tf/linalg) .
- O SVD também pode ser aplicado para construir [sistemas de recomendação](https://developers.google.com/machine-learning/recommendation/labs/movie-rec-programming-exercise) .

Para obter mais exemplos de uso das APIs Core do TensorFlow, confira o [guia](https://www.tensorflow.org/guide/core) . Se você quiser saber mais sobre como carregar e preparar dados, consulte os tutoriais sobre [carregamento de dados de imagem](https://www.tensorflow.org/tutorials/load_data/images) ou [carregamento de dados CSV](https://www.tensorflow.org/tutorials/load_data/csv).